In [1]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

if os.path.join('..', '..', 'recommenders') not in sys.path:
    sys.path.append(os.path.join('..', '..', 'recommenders'))

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


# Locally import the model
from models.deeprec.models.sequential.din import DIN_RECModel as SeqModel


#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

import pickle as pkl
from recommenders.models.deeprec.deeprec_utils import load_dict

System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Tensorflow version: 2.9.0


In [2]:
yaml_file = '../../recommenders/models/deeprec/config/din.yaml'  

In [3]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "..", "tests", "resources", "deeprec", "slirec")

In [4]:

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
# sample_rate = 0.01 # sample a small item set for training and testing here for fast example
sample_rate = 1

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

user_vocab: 123960

item_vocab: 50024

cate_vocab: 164

hist_length:

In [5]:
from models.deeprec.deeprec_utils import load_dict

import pandas as pd

In [6]:
user_vocab = load_dict(user_vocab)
item_vocab = load_dict(item_vocab)
cate_vocab = load_dict(cate_vocab)

In [7]:
print(len(user_vocab), len(item_vocab), len(cate_vocab))

123960 50024 164


In [8]:
lens = []
with open(train_file, 'r') as f:
    for line in f.readlines():
        lens.append(len(line.strip().split('\t')[5].strip().split(",")))

In [10]:
with open(train_file, 'r') as f:
    for line in f.readlines():
#         lens.append(len(line.strip().split('\t')[5].strip().split(",")))
        print(line)
        break

1	ADZPIG9QOCDG5	6300251004	Movies	1138752000	0780623746	Movies	1138752000



In [9]:
df = pd.DataFrame(lens)
df.describe()

,0
count,1.325653e+06
mean,7.972492e+01
std,2.091789e+02
min,1.000000e+00
25%,3.000000e+00
50%,1.100000e+01
75%,4.900000e+01
max,2.365000e+03


In [9]:
inpute_iterator = SequentialIterator()
for line in inpute_iterator.load_data_from_file(train_file):
    print(line)
    break

TypeError: __init__() missing 2 required positional arguments: 'hparams' and 'graph'

In [5]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/din_dice/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/din_dice/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
                          attention_mode="inner_product",
                          activation=['dice', 'dice'],
                          dice_momentum=0.9
            )

In [6]:
hparams.values()

{'use_entity': True,
 'use_context': True,
 'cross_activation': 'identity',
 'user_dropout': True,
 'dropout': [0.3, 0.3],
 'attention_dropout': 0.0,
 'load_saved_model': False,
 'fast_CIN_d': 0,
 'use_Linear_part': False,
 'use_FM_part': False,
 'use_CIN_part': False,
 'use_DNN_part': False,
 'init_method': 'tnormal',
 'init_value': 0.01,
 'embed_l2': 0.0,
 'embed_l1': 0.0,
 'layer_l2': 0.0,
 'layer_l1': 0.0,
 'cross_l2': 0.0,
 'cross_l1': 0.0,
 'reg_kg': 0.0,
 'learning_rate': 0.001,
 'lr_rs': 1,
 'lr_kg': 0.5,
 'kg_training_interval': 5,
 'max_grad_norm': 2,
 'is_clip_norm': 0,
 'dtype': 32,
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 400,
 'enable_BN': True,
 'show_step': 20,
 'save_model': True,
 'save_epoch': 1,
 'write_tfevents': True,
 'train_num_ngs': 4,
 'need_sample': True,
 'embedding_dropout': 0.0,
 'EARLY_STOP': 10,
 'min_seq_length': 1,
 'slots': 5,
 'cell': 'SUM',
 'user_vocab': '../../tests/resources/deeprec/slirec/user_vocab.pkl',
 'item_vocab': '../../tests/r

In [7]:
input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [8]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

/home/caozheng/Desktop/code/recommenders/examples/00_quick_start/../../recommenders/models/deeprec/models/base_model.py:745: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(
2022-06-10 01:32:24.077349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 01:32:24.080418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 01:32:24.080537: I tensorflow/stream_executor/cuda

step 20 , total_loss: 1.5710, data_loss: 1.5710
step 40 , total_loss: 1.5630, data_loss: 1.5630
step 60 , total_loss: 1.5602, data_loss: 1.5602
step 80 , total_loss: 1.4996, data_loss: 1.4996
step 100 , total_loss: 1.5273, data_loss: 1.5273
step 120 , total_loss: 1.5562, data_loss: 1.5562
step 140 , total_loss: 1.5297, data_loss: 1.5297
step 160 , total_loss: 1.5004, data_loss: 1.5004
step 180 , total_loss: 1.5397, data_loss: 1.5397
step 200 , total_loss: 1.5103, data_loss: 1.5103
step 220 , total_loss: 1.4901, data_loss: 1.4901
step 240 , total_loss: 1.5193, data_loss: 1.5193
step 260 , total_loss: 1.5051, data_loss: 1.5051


KeyboardInterrupt: 

In [ ]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

In [ ]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

In [ ]:
'''
dice + op
{'auc': 0.8221,
 'logloss': 1.0988,
 'mean_mrr': 0.6528,
 'ndcg@2': 0.5977,
 'ndcg@4': 0.6752,
 'ndcg@6': 0.708,
 'group_auc': 0.8229}
 
dice + op + disable bn
{'auc': 0.8192,
 'logloss': 1.1587,
 'mean_mrr': 0.6455,
 'ndcg@2': 0.5885,
 'ndcg@4': 0.6685,
 'ndcg@6': 0.7026, 
 'group_auc': 0.8199}

'''


model_best_trained.run_eval(test_file, num_ngs=test_num_ngs)